In [ ]:
import json
from abc import ABC, abstractmethod

# Base Abstract Class
class Employee(ABC):
    def __init__(self, employee_id: str, name: str, department: str):
        self._employee_id = employee_id
        self._name = name
        self._department = department

    @property
    def employee_id(self):
        return self._employee_id

    @property
    def name(self):
        return self._name

    @property
    def department(self):
        return self._department

    @department.setter
    def department(self, value):
        self._department = value

    @abstractmethod
    def calculate_salary(self) -> float:
        pass

    def display_details(self) -> str:
        return f"ID: {self._employee_id}, Name: {self._name}, Dept: {self._department}"

    def to_dict(self) -> dict:
        return {
            "id": self._employee_id,
            "name": self._name,
            "department": self._department
        }

# Full Time Employee
class FullTimeEmployee(Employee):
    def __init__(self, employee_id, name, department, monthly_salary):
        super().__init__(employee_id, name, department)
        self._monthly_salary = max(0, monthly_salary)

    @property
    def monthly_salary(self):
        return self._monthly_salary

    @monthly_salary.setter
    def monthly_salary(self, value):
        self._monthly_salary = max(0, value)

    def calculate_salary(self):
        return self._monthly_salary

    def display_details(self):
        return f"{super().display_details()}, Salary: {self._monthly_salary}"

    def to_dict(self):
        data = super().to_dict()
        data.update({
            "monthly_salary": self._monthly_salary,
            "type": "fulltime"
        })
        return data

# Part Time Employee
class PartTimeEmployee(Employee):
    def __init__(self, employee_id, name, department, hourly_rate, hours_worked):
        super().__init__(employee_id, name, department)
        self._hourly_rate = max(0, hourly_rate)
        self._hours_worked_per_month = max(0, hours_worked)

    @property
    def hourly_rate(self):
        return self._hourly_rate

    @hourly_rate.setter
    def hourly_rate(self, value):
        self._hourly_rate = max(0, value)

    @property
    def hours_worked_per_month(self):
        return self._hours_worked_per_month

    @hours_worked_per_month.setter
    def hours_worked_per_month(self, value):
        self._hours_worked_per_month = max(0, value)

    def calculate_salary(self):
        return self._hourly_rate * self._hours_worked_per_month

    def display_details(self):
        return f"{super().display_details()}, Rate: {self._hourly_rate}, Hours: {self._hours_worked_per_month}"

    def to_dict(self):
        data = super().to_dict()
        data.update({
            "hourly_rate": self._hourly_rate,
            "hours_worked_per_month": self._hours_worked_per_month,
            "type": "parttime"
        })
        return data

# Manager Class
class Manager(FullTimeEmployee):
    def __init__(self, employee_id, name, department, monthly_salary, bonus):
        super().__init__(employee_id, name, department, monthly_salary)
        self._bonus = max(0, bonus)

    @property
    def bonus(self):
        return self._bonus

    @bonus.setter
    def bonus(self, value):
        self._bonus = max(0, value)

    def calculate_salary(self):
        return super().calculate_salary() + self._bonus

    def display_details(self):
        return f"{super().display_details()}, Bonus: {self._bonus}"

    def to_dict(self):
        data = super().to_dict()
        data.update({
            "bonus": self._bonus,
            "type": "manager"
        })
        return data

# Company Class
class Company:
    def __init__(self, data_file='employees.json'):
        self._employees = {}
        self._data_file = data_file
        self._load_data()

    def _load_data(self):
        try:
            with open(self._data_file, 'r') as f:
                data = json.load(f)
                for emp_data in data:
                    emp_type = emp_data.get("type")
                    if emp_type == "fulltime":
                        emp = FullTimeEmployee(emp_data['id'], emp_data['name'], emp_data['department'], emp_data['monthly_salary'])
                    elif emp_type == "parttime":
                        emp = PartTimeEmployee(emp_data['id'], emp_data['name'], emp_data['department'], emp_data['hourly_rate'], emp_data['hours_worked_per_month'])
                    elif emp_type == "manager":
                        emp = Manager(emp_data['id'], emp_data['name'], emp_data['department'], emp_data['monthly_salary'], emp_data['bonus'])
                    else:
                        continue
                    self._employees[emp.employee_id] = emp
        except FileNotFoundError:
            self._employees = {}

    def _save_data(self):
        with open(self._data_file, 'w') as f:
            json.dump([emp.to_dict() for emp in self._employees.values()], f, indent=4)

    def add_employee(self, employee):
        if employee.employee_id in self._employees:
            return False
        self._employees[employee.employee_id] = employee
        self._save_data()
        return True

    def remove_employee(self, employee_id):
        if employee_id in self._employees:
            del self._employees[employee_id]
            self._save_data()
            return True
        return False

    def find_employee(self, employee_id):
        return self._employees.get(employee_id)

    def calculate_total_payroll(self):
        return sum(emp.calculate_salary() for emp in self._employees.values())

    def display_all_employees(self):
        for emp in self._employees.values():
            print(emp.display_details())

    def generate_payroll_report(self):
        print("Payroll Report:")
        print("=" * 50)
        for emp in self._employees.values():
            print(f"{emp.employee_id} | {emp.name} | {emp.__class__.__name__} | Salary: {emp.calculate_salary()}")
        print("=" * 50)
        print(f"Total Payroll: {self.calculate_total_payroll()}")

# Console Interface
def main():
    company = Company()

    while True:
        print("\n1. Add Employee\n2. Remove Employee\n3. Display All Employees\n4. Search Employee\n5. Payroll Report\n6. Exit")
        choice = input("Choose an option: ")

        if choice == '1':
            etype = input("Enter type (fulltime/parttime/manager): ").lower()
            eid = input("ID: ")
            name = input("Name: ")
            dept = input("Department: ")

            if etype == 'fulltime':
                salary = float(input("Monthly Salary: "))
                emp = FullTimeEmployee(eid, name, dept, salary)
            elif etype == 'parttime':
                rate = float(input("Hourly Rate: "))
                hours = float(input("Hours per month: "))
                emp = PartTimeEmployee(eid, name, dept, rate, hours)
            elif etype == 'manager':
                salary = float(input("Monthly Salary: "))
                bonus = float(input("Bonus: "))
                emp = Manager(eid, name, dept, salary, bonus)
            else:
                print("Invalid type.")
                continue

            if company.add_employee(emp):
                print("Employee added.")
            else:
                print("Employee with that ID already exists.")

        elif choice == '2':
            eid = input("Enter Employee ID to remove: ")
            if company.remove_employee(eid):
                print("Employee removed.")
            else:
                print("Not found.")

        elif choice == '3':
            company.display_all_employees()

        elif choice == '4':
            eid = input("Enter Employee ID to search: ")
            emp = company.find_employee(eid)
            if emp:
                print(emp.display_details())
            else:
                print("Not found.")

        elif choice == '5':
            company.generate_payroll_report()

        elif choice == '6':
            break

        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()
